# 2.3.3 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot, Chain-Of-Thought
* Best results for text_prompts: A2, B1, C1, D1
* Best results for img_prompts: B2, C2, D2, E3

### Setup

In [1]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
samantha = OpenAI(instructions)
parser = Parser()
runner = Runner()

In [2]:
pdf_path = "../data/raw/bpläne/3_xplanung/BP_872A_AUFST_TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)
print("Seitenzahl:", len(pdf_prompts))

Seitenzahl: 66


### A) OCR komplettes PDF

In [3]:
instruction = 'Extrahiere den kompletten Textinhalt. Output im LaTeX-Format.'
instruction_prompt = parser.text2prompts([instruction])

In [ ]:
# A1) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Komplettes PDF auf einmal einlesen. Ziel: Konsistentes Layout.
pdf_text = samantha.request([*pdf_prompts, *instruction_prompt])
pprint(pdf_text)

In [ ]:
# A2) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Jede Seite einzeln verarbeiten. Ziel: Fokus maximieren und Context-Window klein halten + Seitenzahlen beleiben erhalten.
prompt_chain = list(map(lambda prompt: samantha.lambdaRequest([prompt, *instruction_prompt]), pdf_prompts))
msgbp872apdf = await asyncio.gather(*prompt_chain)
pprint(msgbp872apdf)
%store msgbp872apdf

### B) Art der baulichen Nutzung

1. text_prompts
2. img_prompts

In [9]:
%store -r msgbaunvo_art msgbp872apdf
context_art = parser.text2prompts([msgbaunvo_art])

In [5]:
# B1) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung".'
results = samantha.similaritySearchWithContext(instruction, msgbp872apdf, context_art)
pprint(len(results[1])) # retrieved documents
pprint(results[0]) # message

25
Hier sind alle Informationen zum Thema "Art der baulichen Nutzung" aus dem bereitgestellten Text:

### \subsection*{C.2. Planungsrechtliche Festsetzungen durch Text}

#### \paragraph{§ 4 Art der baulichen Nutzung}
(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

**Zulässig sind:**
- Wohngebäude,
- die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.

**Nicht zulässig sind:**
- Betriebe des Beherbergungsgewerbes,
- sonstige nicht störende Gewerbebetriebe,
- Anlagen für Verwaltungen,
- Gartenbaubetriebe,
- Tankstellen.

### \subsection*{D.5.2. Begründung einzelner Festsetzungen}

#### \subsection*{D.5.2.1. Art der baulichen Nutzung}

##### \subsubsection*{D.5.2.1.1. Allgemeines Wohngebiet}
Nachdem das aktuelle Planungskonzept vorwiegend ei

In [6]:
# B2) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten

# instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zum Thema "Art der baulichen Nutzung" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'
# msg_pages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_art], img_type="pdf")
# pprint(msg_pages)

pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung".'
msg233_art = samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_art)
pprint(msg233_art)
%store msg233_art

Die Informationen zum Thema "Art der baulichen Nutzung" aus dem Bebauungsplan sind wie folgt:

### § 4 Art der baulichen Nutzung

(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

#### Zulässig sind:
- Wohngebäude,
- die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störenden Handwerksbetriebe,
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.

#### Nicht zulässig sind:
- Betriebe des Beherbergungsgewerbes,
- sonstige nicht störende Gewerbebetriebe,
- Anlagen für Verwaltungen,
- Gartenbaubetriebe,
- Tankstellen.

(2) Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### D.5.2.1. Art der baulichen Nutzung

#### D.5.2.1.1. Allgemeines Wohngebiet

- Das aktuelle Planungskonzept sieht vorwiegend eine wohnbauliche Nutzung innerhalb des Plangebietes vor. 
- Flächen für den Gemeinbedarf mit der Zweckbestimmung

In [14]:
# B3) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF

instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>).'
messages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_art], img_type="pdf")
pprint(messages)

### Art der baulichen Nutzung: 
(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

Zulässig sind:
- Wohngebäude,
- die der Versorgung des Gebietes dienenden Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe,
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.

Nicht zulässig sind:
- Betriebe des Beherbergungsgewerbes,
- sonstige nicht störende Gewerbebetriebe,
- Anlagen für Verwaltungen,
- Gartenbaubetriebe,
- Tankstellen.

(2) Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Referenzen: 6
#############################################


### C) Maß der baulichen Nutzung

1. text_prompts
2. img_prompts

In [10]:
%store -r msgbaunvo_maß msgbp872apdf
context_maß = parser.text2prompts([msgbaunvo_maß])

In [9]:
# C1) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung".'
results = samantha.similaritySearchWithContext(instruction, msgbp872apdf, context_maß)
pprint(len(results[1])) # retrieved documents
pprint(results[0]) # message

25
**Maß der baulichen Nutzung**

Das Maß der baulichen Nutzung wird im allgemeinen Wohngebiet durch die Festlegung der höchstzulässigen Grundflächenzahl (GRZ) und der höchstzulässigen Geschossfläche (GF), durch die Zahl der Vollgeschosse (Höchstmaß bzw. zwingend) und durch die Höhe der baulichen Anlagen (OK, Höchstmaß bzw. zwingend) innerhalb der einzelnen überbaubaren Grundstücksflächen (Baufelder) ausreichend bestimmt.

**§ 5 Maß der baulichen Nutzung**

1. Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.
2. In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:
    - WA1: GRZ maximal 0,65
    - WA2: GRZ maximal 0,80
    - WA3: GRZ maximal 0,75
    - WA4: GRZ maximal 0,75

**§ 6 Höhenlage, Bezugspunkte**

Mit der Zulässigkeit einer Überschreitung der festgesetzten Gebäudeoberkanten (OK) durch

In [10]:
# C2) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten

# instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zum Thema "Maß der baulichen Nutzung" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'
# msg_pages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
# pprint(msg_pages)

pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung".'
msg233_maß = samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_maß)
pprint(msg233_maß)
%store msg233_maß

Hier sind alle Informationen zum Thema "Maß der baulichen Nutzung" aus dem Bebauungsplan:

### § 5 Maß der baulichen Nutzung

(1) Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.

(2) In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:
- WA1: GRZ maximal 0,65
- WA2: GRZ maximal 0,80
- WA3: GRZ maximal 0,75
- WA4: GRZ maximal 0,75

### D.5.2.1.2 Maß der baulichen Nutzung

Das Maß der baulichen Nutzung wird im allgemeinen Wohngebiet durch die Festlegung der höchstzulässigen Grundflächenzahl (GRZ) und der höchstzulässigen Geschossfläche (GF), durch die Zahl der Vollgeschosse (Höchstmaß bzw. zwingend) und durch die Höhe der baulichen Anlagen (OK, Höchstmaß bzw. zwingend) innerhalb der einzelnen überbaubaren Grundstücksflächen (Baufelder) ausreichend bestimmt.

Im WA1, 3 und 4 wird eine höchstzulä

### D) Bauweise, überbaubare Grundstücksflächen

1. text_prompts
2. img_prompts

In [11]:
%store -r msgbaunvo_bg msgbp872apdf
context_bg = parser.text2prompts([msgbaunvo_bg])

In [12]:
# D1) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen".'
results = samantha.similaritySearchWithContext(instruction, msgbp872apdf, context_bg)
pprint(len(results[1])) # retrieved documents
pprint(results[0]) # message

25
### Bauweise, überbaubare Grundstücksflächen

#### § 22 Bauweise
(1) Im Bebauungsplan kann die Bauweise als offene oder geschlossene Bauweise festgesetzt werden.  
(2) In der offenen Bauweise werden die Gebäude mit seitlichem Grenzabstand als Einzelhäuser, Doppelhäuser oder Hausgruppen errichtet.  
(3) In der geschlossenen Bauweise werden die Gebäude ohne seitlichen Grenzabstand errichtet.

#### § 23 Überbaubare Grundstücksfläche
(1) Die überbaubaren Grundstücksflächen können durch die Festsetzung von Baugrenzen, Baulinien oder Bebauungstiefen bestimmt werden.  
(2) Ist eine Baulinie festgesetzt, so muss auf dieser Linie gebaut werden.  
(3) Ist eine Baugrenze festgesetzt, so dürfen Gebäude und Gebäudeteile diese nicht überschreiten.

#### D.5.2.3.2 Bauweise, überbaubare Grundstücksflächen

1. Sofern erforderlich, ist die Bauweise in der Planzeichnung (Teil A) festgesetzt.
2. Die überbaubaren Grundstücksflächen sind durch Baulinien und Baugrenzen in der Planzeichnung (Teil A) festge

In [13]:
# D2)Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten

# instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'
# msg_pages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
# pprint(msg_pages)

pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen".'
msg233_bg = samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_bg)
pprint(msg233_bg)
%store msg233_bg

### § 7 Bauweise, Überbaubare Grundstücksflächen, Abstandsflächen

(1) Sofern erforderlich, ist die Bauweise in der Planzeichnung (Teil A) festgesetzt.

(2) Die überbaubaren Grundstücksflächen sind durch Baulinien und Baugrenzen in der Planzeichnung (Teil A) festgesetzt.

(3) Vorbauten dürfen die Baugrenzen bis zu 1,5 m auf einer Breite von maximal 5,0 m je Eingang überschreiten. Balkone dürfen mit Ausnahme des jeweils obersten Geschosses die Baugrenzen bis zu 0,90 m auf einer Breite von maximal 7,50 m je Balkon überschreiten. Zwischen den einzelnen Balkonen muss ein Abstand von mindestens 3,25 m eingehalten werden.

(4) Terrassen an Gebäuden dürfen sich bis zu einer Tiefe von 2,5 m auch auf Flächen außerhalb der überbaubaren Grundstücksflächen erstrecken. Sie dürfen außerhalb der überbaubaren Grundstücksflächen nicht durch Wintergärten oder ähnliches überbaut werden.

(5) Im Kronenbereich der in der Planzeichnung (Teil A) dargestellten und festgesetzten Bestandsgehölze sind die in Abs

### E) Alle Themen in einem Prompt

1. text_prompts (TODO)
2. img_prompts

In [12]:
%store -r msgbaunvo msgbp872apdf
context_all = parser.text2prompts([msgbaunvo])

In [5]:
# E1) Textinformationen: Alle Themen – WITH CONTEXT – IMG_PROMPTS

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'
messages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_all], img_type="pdf")
pprint(messages)

### Art der baulichen Nutzung

**§ 4 Art der baulichen Nutzung**
1. Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.
   - **Zulässig sind:**
     - Wohngebäude
     - Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe, die der Versorgung des Gebietes dienen
     - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke
   - **Nicht zulässig sind:**
     - Betriebe des Beherbergungsgewerbes
     - Sonstige nicht störende Gewerbebetriebe
     - Anlagen für Verwaltungen
     - Gartenbaubetriebe
     - Tankstellen

2. Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Maß der baulichen Nutzung

**§ 5 Maß der baulichen Nutzung**
1. Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.
2. In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen de

In [6]:
# E2) Textinformationen: Alle Themen – WITHOUT CONTEXT – IMG_PROMPTS
# Problem: Kontext ist sehr groß, sodass GPT den Überblick verliert und Informationen verliert. 

instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>)'
messages = await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
pprint(messages)

### Art der baulichen Nutzung:
- Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.
- Zulässig sind:
  - Wohngebäude
  - Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe, die der Versorgung des Gebietes dienen
  - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke
- Nicht zulässig sind:
  - Betriebe des Beherbergungsgewerbes
  - sonstige nicht störende Gewerbebetriebe
  - Anlagen für Verwaltungen
  - Gartenbaubetriebe
  - Tankstellen
  - Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Maß der baulichen Nutzung:
- Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.
- In den allgemeinen Wohngebieten darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:
  - WA1: GRZ 

In [7]:
# TODO WITH CONTEXT!!!

# E3) Ein Thread – Relevante Seiten extrahieren (Vorverarbeitung) – (Bestes Ergebnis) – WITHOUT CONTEXT
# Wenn Dokument zu groß wird: z.B. chunk_size=10 pages, overlap=2 pages.
# chunk_size = 10, overlap = 2
# n = math.ceil(len(pdf_prompts)/chunk_size)
# chunks = list(map(lambda i: pdf_prompts[slice(max(i*chunk_size-overlap,0),i*chunk_size+chunk_size)], range(n)))
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zu folgenden Themen beinhalten: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'

msg_pages = await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
pprint(msg_pages)
pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all

# instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche.'
# msg233_all = samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_all)
# pprint(msg233_all)
# %store msg233_all

```json
{
  "Seiten": [6, 7, 8, 28, 29, 30, 31, 32, 33]
}
```


In [8]:
# E4) Mehrere Threads – WITHOUT CONTEXT
# Idee: Jede Seite einzeln verarbeiten, um Context-Window klein zu halten.
# Problem: Informationen sind auf mehrere Seiten verteilt => fehlender Kontext => unvollständige Resultate.
async def run(pdf_prompt):
    return samantha.request([
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Steht keine relevante Information zur Verfügung gebe "-" aus.'
        },
        pdf_prompt
    ])

prompt_chain = list(map(lambda pdf_prompt: run(pdf_prompt), pdf_prompts))
results = await asyncio.gather(*prompt_chain)
pprint(results)

- Art der baulichen Nutzung: -
- Maß der baulichen Nutzung: -
- Bauweise: -
- überbaubare Grundstücksfläche: -
#############################################
Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Inhaltsverzeichnis des Bebauungsplans:

- **Art der baulichen Nutzung**: -
- **Maß der baulichen Nutzung**: -
- **Bauweise**: -
- **Überbaubare Grundstücksfläche**: -

Das Inhaltsverzeichnis enthält keine spezifischen Informationen zu diesen Themen. Um detaillierte Informationen zu erhalten, müsste der vollständige Text des Bebauungsplans eingesehen werden.
#############################################
Die angeforderte Seite enthält keine Informationen zu den Themen "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise" und "überbaubare Grundstücksfläche". Es handelt sich um eine Seite mit Abkürzungen und deren Erläuterungen.

- Art der baulichen Nutzung: -
- Maß der baulichen Nutzung: -
- Bauweise: -
- überbaubare Grundstücksfläche: -
######